In [4]:
import allel
import zarr
import numcodecs
import numpy as np
import sys
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter
import argparse

In [5]:
# parse the command line arguements
# - validate the arguements 
# - throw errors if requiremabents are missing
# - validate the filter strings

#pi_dxy --pi --dxy\ # must include at least 1 of pi and/dxy 
#--vcf allsites.vcf.gz \ # required
#--zarr path/to/zarr \  # default to the vcf folder
#--populations popfile.txt \ # only required for dxy
#--window_size 10000 \ # not required, defaults to whole genome
#--snp_filter_expression "DP>=10, GQ>=20, FS>2" \ # required
#--monomorphic_filter_expression "DP>=10, RGQ>=20" \ # required
#--out pi_dxy_out.txt # default to vcf path + suffix

# initialize and add arguments to the argparser

parser = argparse.ArgumentParser(description='pixy: senisbly calculate pi and/or dxy from a VCF containing variant and invariant sites')

parser.add_argument('--version', action='version', version='%(prog)s 1.0')
parser.add_argument('--stats', choices=['pi', 'dxy', 'pi_dxy'], help='Which stats to to calulate from the VCF (pi, dxy, or both)', required=True)
parser.add_argument('--vcf', type=str, nargs='?', help='Path to the input VCF', required=True)
parser.add_argument('--zarr', type=str, nargs='?', help='Folder in which to build the Zarr database', required=True)
parser.add_argument('--populations', type=str, nargs='?', help='Path to the populations file')
parser.add_argument('--window_size', type=int, nargs='?', help='Window size in base pairs over which to calculate pi/dxy')
parser.add_argument('--snp_filter_expression', type=str, nargs='?', help='A comma separated list of filters (e.g. DP>=10,GQ>=20) to apply to SNPs', required=True)
parser.add_argument('--invariant_filter_expression', type=str, nargs='?', help='A comma separated list of filters (e.g. DP>=10,RGQ>=20) to apply to invariant sites', required=True)
parser.add_argument('--out', type=str, nargs='?', help='Path to the output file')

#parser.print_help()

# pull out varizbles from the parsed args
args = parser.parse_args('--stats pi_dxy --vcf test.vcf --zarr test/path/ --populations path/to/popfile.txt --snp_filter_expression DP>=10,GQ>=20,FS>2 --invariant_filter_expression DP>=10,RGQ>=20 --out pi_out.txt'.split())
# sim_args = parser.parse_args()

if (args.populations is None) and ((args.stats == 'dxy' or args.stats == 'pi_dxy')):
    parser.error("--stats dxy and --stats pi_dxy requires --populations path/to/popfile.txt")

In [6]:
print(args)

print(args.invariant_filter_expression)

Namespace(invariant_filter_expression='DP>=10,RGQ>=20', out='pi_out.txt', populations='path/to/popfile.txt', snp_filter_expression='DP>=10,GQ>=20,FS>2', stats='pi_dxy', vcf='test.vcf', window_size=None, zarr='test/path/')
DP>=10,RGQ>=20


In [ ]:
# validating inputs

# STEP 1 checking the vcf:
# - check if sci-kit allele can do this
# - check for contig info
# - alternatively use position vector
# - check for invariant sites and throw and error if they dont exist

# STEP 2 validate the population file
# - format is IND POP
# - throw an error if individuals are missing from VCF



In [ ]:
# filtering the data 

# - extracting the filtration values from the filter strings
# - applying the filter to the whole dataset
# - also filter out biallelic snps
# - check to see there is any data left
# - print warning for low data* think more about this


In [7]:
#Test data:
chromosome = "chrX"
vcf_path = 'data/vcf/ag1000/chrX_36Ag_allsites.vcf.gz'
zarr_path = 'data/vcf/ag1000/chrX_36Ag_allsites.zarr'
#vcf_path = '/Users/Katharine Korunes/Documents/Dxy_test_data/chrX_36Ag_allsites.vcf.gz'
#zarr_path = '/Users/Katharine Korunes/Documents/Dxy_test_data/chrX_36Ag_allsites.zarr'
# inspect the zarr data
callset = zarr.open_group(zarr_path, mode='r')
callset.tree(expand=True)

/
 └── chrX
     ├── calldata
     │   ├── AD (23302924, 36, 4) int16
     │   ├── DP (23302924, 36) int16
     │   ├── GQ (23302924, 36) int8
     │   ├── GT (23302924, 36, 2) int8
     │   ├── MIN_DP (23302924, 36) int32
     │   ├── PGT (23302924, 36) object
     │   ├── PID (23302924, 36) object
     │   ├── PL (23302924, 36, 3) int32
     │   ├── RGQ (23302924, 36) int32
     │   └── SB (23302924, 36, 4) int32
     ├── samples (36,) object
     └── variants
         ├── AC (23302924, 3) int32
         ├── AF (23302924, 3) float32
         ├── ALT (23302924, 3) object
         ├── AN (23302924,) int32
         ├── BaseQRankSum (23302924,) float32
         ├── CHROM (23302924,) object
         ├── DP (23302924,) int32
         ├── DS (23302924,) bool
         ├── END (23302924,) int32
         ├── ExcessHet (23302924,) float32
         ├── FILTER_LowQual (23302924,) bool
         ├── FILTER_PASS (23302924,) bool
         ├── FS (23302924,) float32
         ├── ID (23302924,) object
         ├── InbreedingCoeff (23302924,) float32
         ├── MLEAC (23302924, 3) int32
         ├── MLEAF (23302924, 3) float32
         ├── MQ (23302924,) float32
         ├── MQRankSum (23302924,) float32
         ├── POS (23302924,) int32
         ├── QD (23302924,) float32
         ├── QUAL (23302924,) float32
         ├── RAW_MQ (23302924,) float32
         ├── REF (23302924,) object
         ├── ReadPosRankSum (23302924,) float32
         ├── SOR (23302924,) float32
         ├── altlen (23302924, 3) int32
         ├── is_snp (23302924,) bool
         └── numalt (23302924,) int32

In [8]:
# For accessing just the GT calls
gt_zarr = callset[chromosome + '/calldata/GT']
gt = allel.GenotypeArray(gt_zarr)

In [9]:
# This method (VariantChunkedTable) works, but is obselete -- trying to get new method working
variants = allel.VariantChunkedTable(callset[chromosome]['variants'], 
                                    names=['POS', 'REF', 'ALT', 'DP', 'FS','MQ', 'QD', 'is_snp','numalt'],
                                     index='POS')

# This seems like it should work, but hasn't worked with any of the types of input we tried:        
#variants = allel.VariantTable(callset)

variants

<VariantChunkedTable shape=(23302924,) dtype=[('POS', '<i4'), ('REF', 'O'), ('ALT', 'O', (3,)), ('DP', '<i4'), ('FS', '<f4'), ('MQ', '<f4'), ('QD', '<f4'), ('is_snp', '?'), ('numalt', '<i4')]
   nbytes=1.2G cbytes=163.7M cratio=7.7
   values=zarr.hierarchy.Group>

In [12]:
#Parse these variables from the CLI
snpDP = 10 #DP>=10, 
snpGQ = 20 #GQ>=20
snpFS = 2 #FS>2
refDP = 10 #DP>=10 - monomorphic sites
refRGQ = 20 #RGQ>=20 - monomorphic sites

#Step 1: Use a boolean array to parse variants from monomorphic sites, so we can filter them separately
snp_select = variants.eval('is_snp')[:]
snp_select

1544262

In [15]:
# Step 2: Use the above step to separate invariant and variant sites
# Variant sites are where snp_select = True
variants_only = gt.compress(snp_select, axis=0)
variants_only
# ideally, the compress() method should work on the VariantTable (just as it works on the GT data) -- having trouble with this

<GenotypeArray shape=(1544262, 36, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/1 ... 0/0 0/0 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/1 0/0 0/1 0/1 0/0
...
./. ./. 1/1 ./. ./. ... ./. ./. 1/1 1/1 ./.
./. ./. ./. 0/0 ./. ... ./. ./. ./. ./. ./.
./. ./. ./. ./. ./. ... ./. ./. ./. ./. ./.

In [ ]:
# Invariant sites are where snp_select = False
ref_only = gt.compress(~snp_select, axis=0)
ref_only

In [ ]:
# Step 3: Once variant/invariant are separated, apply filter expressions

# We can use "numalt" to check if biallelic!
snp_filter_expression = "(DP > {}) & (FS > {}) & (numalt < 2)".format(snpDP,snpFS)
snp_filter_expression
# We can evaluate complex filter expressions if we get the VariantTable format working:
#variant_selection = variants.eval(snp_filter_expression)[:]
#variant_selection

In [ ]:
# calculate pi

# for loop wrapper for pi calcuation
# output looks like:

# chromosome pos1 pos2 no_differences no_comparisons no_missing pi_1 pi_2 dxy
# chromosome pos1 pos2 no_differences no_comparisons no_missing pi
# chromosome pos1 pos2 no_differences no_comparisons no_missing dxy

# check if pi output makes sense
# - are any pis/dxys all zero?
# - check if # missing == (pos2 - pos1)
# - check if everything was either compared or missing
# - write out summary of program parameters file* think about how this should work


In [ ]:
#Basic functions for comparing the genotypes at each site in a region: counts differences out of sites with data

#For the given region: return average pi, # of differences, # of comparisons, and # missing.
# this function loops over every site in a region passed to it
def tallyRegion(gt_region):
    total_diffs = 0
    total_comps = 0
    total_missing = 0
    for site in gt_region:
        vec = site.flatten()
        #now we have an individual site as a numpy.ndarray, pass it to the comparison function
        site_diffs, site_comps, missing = compareGTs(vec)
        total_diffs += site_diffs
        total_comps += site_comps
        total_missing += missing
    if total_comps > 0:
        avg_pi = total_diffs/total_comps
    else:
        avg_pi = 0
    return(avg_pi, total_diffs, total_comps, total_missing)

#Out of sites with data, count differences. 
#Return the number of differences, the number of comparisons, and missing data count.
def compareGTs(vec):
    # use gts to select only sites with data. 
    # counting missing data as a check, but it's implicit in the length of (gts)
    gts = []
    missing = 0
    for x in vec:
        if x in (0,1):
            gts.append(x)
        else:
            missing += 1
    c = Counter(gts)
    #print(c)
    diffs = c[1]*c[0]
    comps = len(list(combinations(gts, 2)))        
    return(diffs,comps,missing)

In [ ]:
# Compute pi over a chosen window size
pos = allel.SortedIndex(callset[chromosome + '/variants/POS'])

#window size:
w = 1000

#Testing with a 10kb chunk, but can modify later to default to chromosome length
y = w
for x in range (1, 10000, w):
    loc_region = pos.locate_range(x, y)
    gt_region1 = allel.GenotypeArray(gt_zarr[loc_region])
    avg_pi, total_diffs, total_comps, total_missing = tallyRegion(gt_region1)
    print("Region:", x , y, ",", "Region length:", len(gt_region1))
    print("Average pi:", avg_pi)
    print("Diffs, comps, missing:", total_diffs, total_comps, total_missing, "\n")
    y += w

In [5]:
# convert this notebook to a .py script
!jupyter nbconvert --to=python pixy.ipynb

[NbConvertApp] Converting notebook pixy.ipynb to python
[NbConvertApp] Writing 4002 bytes to pixy.py
